In [1]:
from elasticsearch import Elasticsearch
es=Elasticsearch([{'host':'localhost','port':9200}])

In [2]:
!pip install pattern

Processing c:\users\markz\appdata\local\pip\cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e\future-0.18.2-cp37-none-any.whl
  Using cached lxml-4.5.0-cp37-cp37m-win_amd64.whl (3.7 MB)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332728 sha256=02a79c4d3ce2f4051a7d7fcfcfeab87a4434049a7f5132a9bc9c3701c2b5ef72
  Stored in directory: c:\users\markz\appdata\local\pip\cache\wheels\8d\1f\4e\9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44944 sha256=060fd5a1a482371271b10507efada5e5f9d414c7fc7b621d2bf72358755e5641
  Stored in directory: c:\users\markz\appdata\local\pip\cache\wheels\4a\87\a9\ef99b7b68f9816d5ef1262dc11ce31a93a844eb69e68c1a031
  Created wheel for nltk: filename=nltk-3.5-py3-no

In [3]:
from pattern.en import ngrams
from pattern.en import lemma

In [6]:
import csv
reader = csv.reader(open('dict.csv', 'r',encoding="utf8"))
pharma_dict = dict((rows[0],rows[1]) for rows in reader)

In [31]:
res=es.get(index='posts',doc_type='_doc', id = 'a0fab5b4b83ed406ac4f97859a285ad4')

In [32]:
res

{'_index': 'posts',
 '_type': '_doc',
 '_id': 'a0fab5b4b83ed406ac4f97859a285ad4',
 '_version': 1,
 '_seq_no': 2418,
 '_primary_term': 1,
 'found': True,
 '_source': {'keyword': 'cổ phiếu MSN',
  'url': 'https://ndh.vn/co-phieu/thi-truong-a-u-re-e-hap-dan-nha-au-tu-tro-lai-1024885.html',
  'title': 'Thị trường đã đủ rẻ để hấp dẫn nhà đầu tư trở lại?',
  'content': '&nbsp;Như vậy là sau khi tăng điểm trong tuần lễ từ ngày 3/3 đến 11/3 từ 452 lên 495 điểm, chỉ số VN-Index đã giảm trong nhiều phiên sau đó và giảm còn 464 điểm trong ngày hôm 22/3.Việc này thể hiện giằng co giữa một bên là tâm lý lạc quan trước chính sách đúng đắn của chính phủ và một bên là nỗi lo lắng về những hệ quả mà chính sách thắt chặt có thể mang lại.Chúng tôi nhận thấy rằng lòng tin của nhà đầu tư vào chính sách và ổn định vĩ mô đang tăng lên đáng kể, rằng lạm phát và bất ổn tỷ giá rồi sẽ được giải quyết.Tuy nhiên, một câu hỏi đặt ra là quá trình này sẽ mất bao lâu? Trong khi vấn đề này chưa thể có câu trả lời rõ rà

In [33]:
final_text = res['_source']['tokenize_content']

In [34]:
text1 = ngrams(final_text, n=1, punctuation=".,;:!?()[]{}`''\"@#$^&*+-|=~_", continuous=False)
final_text2 = ngrams(final_text, n=2, punctuation=".,;:!?()[]{}`''\"@#$^&*+-|=~_", continuous=False)
final_text3 = ngrams(final_text, n=3, punctuation=".,;:!?()[]{}`''\"@#$^&*+-|=~_", continuous=False)

In [35]:
new_dict = {}
new_dict1 = {}
new_dict2 = {}
positive_score,negative_score = 0,0
final_text1 = []

In [36]:
for x in text1:
    final_text1.append(lemma(x[0]))

In [37]:
for k,v in pharma_dict.items():
    for x in final_text1:
        if x == k:
            new_dict[x] = pharma_dict[x] 

In [38]:
for k,v in pharma_dict.items():
    for x in final_text2:
        temp = x[0] + ' ' + x[1]
        if temp == k:
            new_dict1[x] = pharma_dict[temp]

In [39]:
for k,v in pharma_dict.items():
    for x in final_text3:
        temp = x[0] + ' ' + x[1] + ' ' + x[2]
        if temp == k:
            new_dict2[x] = pharma_dict[temp]  

In [40]:
print(new_dict, new_dict1)

{'nhu_cầu': 'positive', 'hỗ_trợ': 'positive', 'khó_khăn': 'negative', 'giảm_giá': 'negative'} {}


In [41]:
positive_list = [] ; negative_list = [];

In [42]:
for key, value in new_dict.items():
    if value == 'positive':
        positive_list.append(key)
    if value == 'negative':
        negative_list.append(key)

In [43]:
for key, value in new_dict1.items():
    if value == 'positive':
        positive_list.append(key)
    if value == 'negative':
        negative_list.append(key)

In [44]:
positive_score = len(positive_list) ; negative_score = len(negative_list);

In [45]:
overall_score = positive_score - negative_score
decision = ''

In [46]:
if overall_score < 0:
    decision = 'sell'
elif overall_score == 0:
    decision = 'hold'
else:
    decision = 'buy'

In [47]:
print(overall_score)

0
